In [58]:
import sys
sys.path.append('../')

In [59]:
# forcing the notebook to reload the modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
from src.data import datacollector as dc
from src.data import downloader as dl
from src.data import datafilter as df

In [61]:
PATH_DB_CONFIGS = '../data/'
PATH_URL_LISTS = '../data/raw/url_lists/'
PATH_RAW = '../data/raw/'
PATH_PROCESSED = '../data/processed/'

In [62]:
# Download files from Databases
GOOGLEFONTSDB = True
DSRFONTSDB = True

dl.get_font_dbs({"GoogleFontsDB" : GOOGLEFONTSDB,
              "DSRFontsDB" : DSRFONTSDB}, 
             PATH_DB_CONFIGS,
             PATH_RAW)

Accessing GoogleFontsDB.
3 directories for transfer.
Transferring files...
File transfer successful
Accessing DSRFontsDB.
Loaded key for repository access.
4 directories for transfer.
Transferring files...
File transfer successful


In [6]:
# Process link lists, starting with glyphazzn
dl.update_glyphazzn_list(PATH_RAW, PATH_URL_LISTS)

# Start download. This will take a while! (approx 80 mins for glyphazzn)
dl.download_files_from_txts(PATH_URL_LISTS, PATH_RAW)

glyzphazzn link list updated.
glyphazzn_raw.txt successfully extracted.
Processing glyphazzn.txt...
Checking 66260 URLs...


In [63]:
# Go through all folders of raw database data
# collect all fonts, write description to csv
# dump all files into a single folder

dc.collectfonts(PATH_RAW)



Unpacking all zip files...
Converting all OTF files to TTF...


CalledProcessError: Command '['python', 'path/to/ttf-converter.py', '--input-dir', '../data/raw/', '--output-dir', '../data/processed/']' returned non-zero exit status 2.

In [ ]:
df.filter_fonts("../data/processed/all-fonts", "../data/processed/filtered-fonts")